In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
import requests
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [56]:
# Config

# Create Backup
# --------------------

os.system("mkdir backup")

0

In [89]:
## Richemont Scrapper

richemont_job_db = []

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')
browser.get('https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow=1')

soup = BeautifulSoup(browser.page_source,'lxml')

for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
    row_scrap = []
    job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
    job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
    company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
    departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
    location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()
    
    row_scrap.append(job_title)
    row_scrap.append(job_link)
    row_scrap.append(company)
    row_scrap.append(departement)
    row_scrap.append(location)
    
    richemont_job_db.append(row_scrap)

i = 14
while i <= int(soup.find('a', class_='paginationItemLast')['href'][-3:]) :
    browser.get(f'https://jobs.richemont.com/search/?q=&sortColumn=referencedate&sortDirection=desc&optionsFacetsDD_country=FR&startrow={i}')

    soup = BeautifulSoup(browser.page_source,'lxml')
    
    for x in soup.find_all('tr', attrs={'class' : 'data-row clickable'}):
        row_scrap = []
        job_title = x.find('span', attrs={'class' : 'jobTitle'}).text.replace('\n', '')
        job_link = x.find('span', attrs={'class' : 'jobTitle'}).find('a')['href']
        company = x.find('span', attrs={'class' : 'jobFacility'}).text.replace('\n', '')
        departement = x.find('span', attrs={'class' : 'jobDepartment'}).text.replace('\n', '')
        location = x.find('span', attrs={'class' : 'jobLocation'}).text.replace('\n', '').strip()

        row_scrap.append(job_title)
        row_scrap.append(job_link)
        row_scrap.append(company)
        row_scrap.append(departement)
        row_scrap.append(location)

        richemont_job_db.append(row_scrap)
    
    i = i + 14
    
richemont_job_db = pd.DataFrame(richemont_job_db)
richemont_job_db = richemont_job_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company",
                                                    3 : "Department", 4 : "Location"})

richemont_job_db_copy = richemont_job_db
richemont_job_db_copy['desc'] = ''

# Add Desc to job offer
# add https://jobs.richemont.com before links to connect

i = 0

while i <= len(richemont_job_db_copy)-1:
    browser.get('https://jobs.richemont.com' + richemont_job_db_copy['link'][i])
    soup = BeautifulSoup(browser.page_source,'lxml')
    richemont_job_db_copy['desc'][i] = soup.find('span', class_="jobdescription").text
    i = i + 1
richemont_job_db_copy.head()

richemont_job_db_copy.to_csv('backup/richemont_db.csv')
richemont_job_db_copy.to_json('backup/richemont_db.json')

In [91]:
## Dassault Aviation

dassaultAviation_db = []

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

i = 1
while i <= 10:
    browser.get(f'https://carriere.dassault-aviation.com/offre-de-emploi/liste-offres.aspx?page={i}&LCID=1036')
    soup = BeautifulSoup(browser.page_source,'lxml')

    for x in soup.find_all('li', class_='ts-offer-list-item'):
            row_scrap = []
            job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
            job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
            company = "Dassault Aviation"
            departement = 'N/A'
            details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(details)

            dassaultAviation_db.append(row_scrap)
    
    i = i + 1

dassaultAviation_db = pd.DataFrame(dassaultAviation_db)
dassaultAviation_db = dassaultAviation_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
dassaultAviation_db_copy = dassaultAviation_db
dassaultAviation_db_copy['desc'] = ''

# Add Desc to job offer
# add https://carriere.dassault-aviation.com before links to connect

i = 0
while i <= len(dassaultAviation_db_copy)-1:
    browser.get('https://carriere.dassault-aviation.com' + dassaultAviation_db_copy['link'][i])
    soup = BeautifulSoup(browser.page_source,'lxml')
    dassaultAviation_db_copy['desc'][i] = soup.find('div', attrs={'id' : 'contenu-ficheoffre'}).text
    i = i + 1

dassaultAviation_db_copy.to_csv('backup/dassaultAviation_db.csv')
dassaultAviation_db_copy.head()

In [ ]:
# Air France

browser = webdriver.Chrome('/Users/maxime_alain/Downloads/chromedriver')

In [96]:
browser.get('https://recrutement.airfrance.com/offre-de-emploi/liste-offres.aspx')

In [106]:
airFrance_db = []

soup = BeautifulSoup(browser.page_source,'lxml')

for x in soup.find_all('li', class_='ts-offer-list-item'):
            row_scrap = []
            job_title = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'}).text.replace('\n', '').strip()
            job_link = x.find('a', attrs={'class' : 'ts-offer-list-item__title-link'})['href']
            company = "Air France"
            departement = 'N/A'
            details = x.find('ul', attrs={'class' : 'ts-offer-list-item__description'}).find_all('li')

            row_scrap.append(job_title)
            row_scrap.append(job_link)
            row_scrap.append(company)
            row_scrap.append(departement)
            row_scrap.append(details)

            airFrance_db.append(row_scrap)

airFrance_db = pd.DataFrame(airFrance_db)
airFrance_db = airFrance_db.rename(columns={0 : "Position", 1 : "link", 2 : "Company", 3 : "Department", 4 : "Location"}) #
airFrance_db

,Position,link,Company,Department,Location
0,STAGE – Développeur Web Full Stack H/F,/offre-de-emploi/emploi-stage-developpeur-web-...,Air France,N/A,"[[Convention de stage], [Occitanie]]"
1,STAGE - Ingénierie Analyse des Paramètres Enre...,/offre-de-emploi/emploi-stage-ingenierie-analy...,Air France,N/A,"[[Convention de stage], [Ile-de-France]]"
2,Alternance – Développeur Front-End H/F,/offre-de-emploi/emploi-alternance-developpeur...,Air France,N/A,"[[Alternance et apprentissage], [Provence-Alpe..."
3,ALTERNANCE – Assistance à Maîtrise d'Ouvrage -...,/offre-de-emploi/emploi-alternance-assistance-...,Air France,N/A,"[[Alternance et apprentissage], [Ile-de-France]]"
4,Juriste HOP! CDD H/F,/offre-de-emploi/emploi-juriste-hop-cdd-h-f_11...,Air France,N/A,"[[CDD], [Pays de la Loire]]"
5,Agent gestion des risques HOP! CDD H/F,/offre-de-emploi/emploi-agent-gestion-des-risq...,Air France,N/A,"[[CDD], [Ile-de-France]]"
6,RESPONSABLE COMPTABILITÉ FOURNISSEURS TRANSAVI...,/offre-de-emploi/emploi-responsable-comptabili...,Air France,N/A,"[[CDI], [Ile-de-France]]"
7,ALTERNANCE – Business Enterprise Architect SI ...,/offre-de-emploi/emploi-alternance-business-en...,Air France,N/A,"[[Alternance et apprentissage], [Ile-de-France]]"
8,STAGE - Assistant Relations Sociales H/F,/offre-de-emploi/emploi-stage-assistant-relati...,Air France,N/A,"[[Convention de stage], [Ile-de-France]]"
9,STAGE - Assistant Méthodes et Développement Op...,/offre-de-emploi/emploi-stage-assistant-method...,Air France,N/A,"[[Convention de stage], [Ile-de-France]]"
